# Automated Conversation between 2 bots

## About Bots
This project accomplishes a back and forth conversation between a flight assistant bot and customer bot. The flight assistant bot is responsible for handling queries related to booking return flights in any European country, while the customer bot is given the task to find the cheapest ticket (with return) to any randomly chosen 5 European countries for a vacation holiday coming soon. You can read the the first 2 system prompts below to get a better overview. 

## Selecting LLMs
After doing a lot of trials, I found out that Anthropic's Claude model performance was not even close to the way Gemini and ChatGPT gave responses, with the same system prompt. Claude's response were empty (None) most of the time, even by swapping the role. If anyone figures out why please let me know (sabeehmehtab@gmail.com), thanks!

## Tool Issues
I did implement the use of tools but for some reason ChatGPT model does not consider using it. Though my implementation of tools is a bit tricky, I have used a separate model (Claude because it failed above) for handling tool calls from a GPT chatting model when it has the role of a flight assistant. This tool handling Claude model receives a query/task input generated from the GPT and is given a further set of tools (3 in this case) to help it answer the query/task. The issue is it never gets till this point. The GPT model never uses it since it can figure out the answer to any query from the customer bot on its own. Just to mention, I did a few tries by changing the system prompt to kind of force it to use tools but did not get any success. 

## Imports

In [0]:
# imports
import os
import json
import time
import random
import anthropic
import gradio as gr
import google.generativeai
from dotenv import load_dotenv
from openai import OpenAI
from datetime import date, timedelta

## Setup keys from environment file

In [0]:
#Load available keys from environment file 
#Print the keys first 6 characters 

load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
ant_api_key = os.getenv("ANTHROPIC_API_KEY")
goo_api_key = os.getenv("GOOGLE_API_KEY")

if openai_api_key:
    print(f"OpenAI API key exists and begins {openai_api_key[:6]}")
else:
    print("OpenAI API key does not exist")

if ant_api_key:
    print(f"Anthropic API key exists and begins {ant_api_key[:6]}")
else:
    print("Anthropic API key API key does not exist")

if goo_api_key:
    print(f"Google API key exists and begins {goo_api_key[:6]}")
else:
    print("Google API key does not exist")

## Model(s) Initialization

In [0]:
# Setup code for OpenAI, Anthropic and Google

openai = OpenAI()
gpt_model = "gpt-4o-mini"

claude_sonnet = anthropic.Anthropic()
claude_model = "claude-3-7-sonnet-latest"

google.generativeai.configure()
gemini_model = "gemini-2.0-flash"

## Define System Prompts

In [0]:
system_prompt1 = "You are a helpful assistant chatbot for an airline called 'Edge Air'.  \
You are to respond to any queires related to booking of flights in European countries. \
You should offer a discount of 10% to European Nationals and a 5% discount on debit/credit card payments, when asked. \
You are provided with a tool that you can use when customer query is related to return ticket price or flight duration or available dates. \
Responses must be in a polite and courteous way, while encouraging the customer to buy a ticket as early as possible."

system_prompt2 = "You are a customer who wants to book a flight at 'Edge Air' airline, via a chatbot assistant. \
You reside in Dubai and will be flying to Europe after 90 days from today on a vacation. \
You are to choose any 5 countries in the European region and find the cheapest return ticket available. \
You should ask for discounts and act smart to get the best available discount.\
Remember to ask questions related to the return flight ticket price, available dates and duration to and from destination city. \
Keep your responses short and precise."

system_prompt3 = "You are an airline flight booking manager who has access to multiple tools required \
in the process of a booking. You will be given a query or task from a chabot assistant that should be responded \
with the help of the tools provided. If no such tool exists to resolve the query/task at hand, \
you must guess the solution and respond back with a high level of confidence. When taking a guess, \
make sure that your solution is relevant to the query/task given by giving a second-thought to it."

starting_prompt = "Start of an autonomous conversation between two AI bots. They take turns for flight booking process discussion."

## Define Flight Assistant tools

In [0]:
# Flight Assistant Tool

def call_manager(task):
    prompt = [
        {"role" : "system", "content" : system_prompt3},
        {"role" : "user", "content" : task}
    ]
    model = "gemini-2.0-flash"
    gemini_via_openai_client = OpenAI(
        api_key=goo_api_key, 
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    response = gemini_via_openai_client.chat.completions.create(model=model,messages=prompt)
    return response.choices[0].message.content


# There's a particular dictionary structure that's required to describe our function:
manager_function = {
    "name": "call_manager",
    "description": "Use this tool only when you are unsure about the answer to the clients query, like when you want to know the ticket price \
    of a country, available traveling dates, duration of the flight journey from one country to another or any other flight booking information ",
    "parameters": {
        "type": "object",
        "properties": {
            "task": {
                "type": "string",
                "description": "The query or task you want to resolve in simple words",
            },
        },
        "required": ["task"],
        "additionalProperties": False
    }
}

assistant_tools = [{"type":"function","function":manager_function}]

## Define Flight Manager Tools

In [0]:
# Flight Manager Tools

fixed_city_durations = {"france":"6 Hours","berlin":"6.5 Hours","germany":"7 Hours","netherlands":"7.5 Hours","spain":"5 Hours"}

def get_ticket_price():
    price = random.randint(800, 2000)
    return price

def get_available_dates():
    available_dates = []
    no_of_dates = random.randint(15,30)
    
    start_date = date.today()
    end_date = start_date + timedelta(180)
    diff = end_date-current_date

    for day in range(no_of_dates):
        random.seed(a=None)
        rand_day = random.randrange(diff.days)
        available_dates.append(current + timedelta(rand_day))

    return available_dates

def get_duration(city):
    city = city.lower()
    if (city in fixed_city_durations.keys()):
        return fixed_city_durations[city]
    else:
        return [f"{random.randint(4,10) + random.random()} Hours", f"{random.randint(4,10) + random.random()} Hours"]
    

### Anthropic tool usage format 

In [0]:
# There's a particular Antrhopic Tool Object structure that's required to describe our tool function for Claude:
price_function = {
    "name":"get_ticket_price",
    "description":"Use this tool to get the price of a return ticket to the destination city. It will return the price in the dollar currency.",
    "input_schema":{
        "type": "object",
        "properties": {},
        "required": []
    }
}
dates_function = {
    "name":"get_available_dates",
    "description":"Use this tool for fetching the available dates of a flight to the destination city. It will return a list of dates that are avilable for travelling.",
    "input_schema":{
        "type": "object",
        "properties": {},
        "required": []
    }
}
duration_function = {
    "name":"get_duration",
    "description":"Use this tool to get the flight durations to and from the destination city. It will return the two flight durations in hours in a string format in a list.",
    "input_schema":{
        "type": "object",
        "properties": {
            "city" : { "type":"String", "description":"Name of the destination city"}
        },
        "required": ["city"]
    }
}

anthropic_manager_tools = [price_function,dates_function,duration_function]

openai_manager_tools = [
    {"type":"function","function":price_function},
    {"type":"function","function":dates_function},
    {"type":"function","function":duration_function}
]


## Gradio Chatbot Conversation Structure

In [0]:
"""
        Commented Claudes conversation chat funtion as it produces a lot of empty responses
"""
def get_structured_messages(history, system_prompt):
    return [{"role" : "system", "content" : system_prompt}] + history

def chat_gpt(system_prompt, history):
    messages = get_structured_messages(history=history, system_prompt=system_prompt)

    response = openai.chat.completions.create(model=gpt_model, messages=messages)#, tools=assistant_tools)

    if (response.choices[0].finish_reason=="tool_calls"):
        message = response.choices[0].message
        response = handle_assistant_tool_call(message)
        messages.append({"role" : "assistant", "content" : messages.content})
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

# def chat_claude(system_prompt, history):    
#     response = claude_sonnet.messages.create(
#         model=claude_model,
#         max_tokens=200,
#         temperature=0.7,
#         system=system_prompt,
#         messages=history,
#     )
#     try:
#         text = response.content[0].text
#     except:
#         print("No response from claude")
#         text = ""
#     return text

def chat_gemini(system_prompt, history):
    gemini = google.generativeai.GenerativeModel(
        model_name=gemini_model,
        system_instruction=system_prompt
    )
    response = gemini.generate_content(json.dumps(history))
    # print(f"Gemini Response: \n{response}")
    return response.text

## Handling Tool Calls

In [0]:
def handle_assistant_tool_call(message):
    content_list = []
    tool_calls = message.tool_calls
    print(f"List of tool call: \n{tool_calls}")
    for tool in tool_calls:
        try:
            arguments = json.loads(tool_call.function.arguments)
        except:
            print("Error loading arguments from tool call")
        print(f"Arguments in json format: \n{arguments}")
        task = arguments.get('task')
        content = run_manager_llm(task)
        content_list.append(content)
    response = {
        "role": "tool",
        "content": content_list,
        "tool_call_id": tool_call.id
    }
    return response
    
# Anthropic Claude-Sonnet
def run_manager_llm(task):
    user_prompt = [
        {"role":"user", "content": task}
    ]

    response = claude_sonnet.messages.create(
        model=claude_model,
        max_tokens=1024,
        tools=anthropic_manager_tools,
        tool_choice='auto',
        temperature=0.7,
        system=system_prompt3,
        messages=user_prompt,
    )

    tool_use = response.content[0].tool_use
    print(f"Claude tool help: {tool_use}")
    
    if tool_use.name=="get_ticket_price":
        price = get_ticket_price()
        response = manager_tool_response(user_prompt,tool_use,price)
    
    elif tool_use.name=="get_available_dates":
        dates = get_available_dates()
        response = manager_tool_response(user_prompt,tool_use,dates)
    elif tool_use.name=="get_duration":
        duration = get_duration(tool_use.input["city"])
        response = manager_tool_response(user_prompt,tool_use,duration)

    try:
        text = response.content[0].text
    except:
        print("No response from claude")
        text = ""
    return text

# Function for generating response after tool usage
def manager_tool_response(user_prompt, tool_use, content):
    user_prompt.append({"role":"assistant","content": [
        {
            "type": "tool_use", "tool_use_id": tool_use.tool_use_id, "name": tool_use.name, "input": tool_use.input,
        }
    ]})
    user_prompt.append({"role":"user","content": [
        {
            "type": "tool_result", "tool_use_id": tool_use.tool_use_id, "content": content,
        }
    ]})
    response = claude_sonnet.messages.create(
        model=claude_model,
        max_tokens=1024,
        tools=anthropic_manager_tools,
        tool_choice='auto',
        temperature=0.7,
        system=system_prompt3,
        messages=user_prompt,
    )
    return response

## Build UI using Gradio

In [0]:
chatbot_models = ["ChatGPT", "Gemini"]

with gr.Blocks() as demo:
    gr.Markdown("# 🤖 AI Chatbot Conversation")
    gr.Markdown("Watch two AI chatbots have a conversation with each other.")
    
    is_conversation_active = gr.State(True)
    turns_count = gr.State(0)
    
    with gr.Row():
        with gr.Column(scale=3):
            # Chat display
            chatbot = gr.Chatbot(
                type='messages',
                label="Bot Conversation",
                height=500,
                elem_id="chatbot",
                avatar_images=("🧑", "🤖",)
            )
            
            # Controls
            with gr.Row(elem_classes="controls"):
                start_btn = gr.Button("Start Conversation", variant="primary")
                stop_btn = gr.Button("Stop", variant="stop")
                clear_btn = gr.Button("Clear Conversation")
            
            # Conversation settings
            with gr.Row():
                max_turns = gr.Slider(
                    minimum=5,
                    maximum=20,
                    value=8,
                    step=1,
                    label="Maximum Conversation Turns",
                    info="How many exchanges between the bots"
                )
                delay = gr.Slider(
                    minimum=1,
                    maximum=5,
                    value=2,
                    step=0.5,
                    label="Delay Between Responses (seconds)",
                    info="Simulates thinking time"
                )
        
        with gr.Column(scale=1):
            gr.Markdown("### About")
            gr.Markdown("""
            This interface simulates a flight booking conversation between two AI chatbots.
            
            - Click "Start Conversation" to begin
            - The bots will automatically exchange messages
            - You can stop the conversation at any time
            
            """)
            bot1 = gr.Dropdown(chatbot_models, show_label=True, label="Flight Assistant Model (left)", multiselect=False)
            bot2 = gr.Dropdown(chatbot_models, show_label=True, label="Customer Model (right)", multiselect=False)

    def bot_response(model, system_prompt, history):
        if model==chatbot_models[0]:
            return chat_gpt(system_prompt=system_prompt,history=history)
        else:
            return chat_gemini(system_prompt=system_prompt,history=history)
    
    # Function to update the conversation display
    def start_conversation(turns, max_turns, delay_time, bot1_model, bot2_model):
        history = []
        conversation = []
        history.append({"role":"user","content":starting_prompt})
        global is_conversation_active
        is_conversation_active=True
        
        while is_conversation_active and turns < max_turns:
            # Airline Assistant Responds first. Change chat function to change bot model 
            message = bot_response(bot1_model,system_prompt1,history=history)
            print(f"(assistant): \n{message}")
            conversation.append({"role":"assistant","content":message})
            history.append({"role":"assistant", "content": message})
            yield conversation, turns 
            time.sleep(delay_time)
            
            # Customer responds next. Change chat function to change bot model 
            reply = bot_response(bot2_model,system_prompt2,history=history)
            print(f"(customer): \n{reply}")
            conversation.append({"role":"user","content":reply})
            history.append({"role":"assistant", "content": reply})
            turns+=1
            yield conversation, turns
            time.sleep(delay_time)
            
    
    # Function to stop the conversation
    def stop_conversation():
        global is_conversation_active
        is_conversation_active=False
        
    
    # Function to clear the conversation
    def clear_conversation():
        global is_conversation_active
        is_conversation_active=False
        return [], 0
    
    # Set up the event handlers
    start_btn.click(
        start_conversation,
        inputs=[turns_count, max_turns, delay, bot1, bot2],
        outputs=[chatbot, turns_count]
    )
    
    stop_btn.click(
        stop_conversation,
        outputs=[]
    )
    
    clear_btn.click(
        clear_conversation,
        outputs=[chatbot, turns_count]
    )
    

In [0]:
demo.launch(share=True)